In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from datetime import datetime

# Load data
data = pd.read_csv('./Data/EURUSD_D1.csv')
data['Time'] = pd.to_datetime(data['Time'], format='%Y-%m-%d %H:%M:%S')
data.set_index('Time', inplace=True)

# Linear Regression Model
class LinearRegressionModel:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        num_samples, num_features = X.shape
        self.weights = np.zeros(num_features)
        self.bias = 0

        for _ in range(self.n_iterations):
            y_pred = np.dot(X, self.weights) + self.bias
            dw = (1 / num_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / num_samples) * np.sum(y_pred - y)
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

# Feature engineering for lag
def featureEngineering(data, target_column, n_past=1):
    X, Y = [], []
    for i in range(n_past, len(data)):
        X.append(data.iloc[i-n_past:i, target_column].values)
        Y.append(data.iloc[i, target_column])
    return np.array(X), np.array(Y)

####################################### Data Preparation and Model Training ############################################
target_column = -2  # Assuming Close price is the target
n_past = 5  # Lookback window size
X, Y = featureEngineering(data, target_column, n_past)

# Train/Test Split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=10, shuffle=False)

Train_Size = len(X_train)   

# Scaling the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train the model
model_lr = LinearRegressionModel(learning_rate=0.01, n_iterations=1000)
model_lr.fit(X_train, Y_train)

################################# # Backtesting Strategy using Linear Regression #########################################

class LinearRegressionStrategy(Strategy):
    def init(self):
       
        # Make predictions on the test data
        self.model= model_lr
        self.predicted_prices = self.model.predict(X_test)
        self.Actual_price = Y_test

        #Plot foreccasted results
       
        i =0
        if i ==0:
            i= i+1
        else:
            i= i+1

        print(f'Init iteration :{i}')

        # Track positions and predictions
        self.index = 0

    def next(self):

        # skip training/ in - sample data
        # if len(self.data) < self.Train_Size:
        #     # print(len(self.data))
        #     return
        

        # We move through test set predicand not self.position.is_long tions
        if self.index < len(self.predicted_prices) - 1:
            pred_price_today = self.predicted_prices[self.index]
            pred_price_tomorrow = self.Actual_price[self.index]

            # Buy if tomorrow's predicted price is higher than today's predicted price
            if pred_price_tomorrow > pred_price_today and not self.position.is_long:
                self.position.close()
                # self.buy()
                self.buy(size=0.11)


            # Sell if tomorrow's predicted price is lower than today's predicted price
            elif pred_price_tomorrow < pred_price_today and not self.position.is_short:
                self.position.close()
                # self.sell()
                self.sell(size=0.11)


            self.index += 1
        
        # print(f'Indx value: {self.index}')

test_data = data.iloc[-len(X_test):]  # This assumes X_test is at the end of the dataset
bt = Backtest(test_data, LinearRegressionStrategy, cash =10000, commission=.002, margin=.05, trade_on_close=False)
stats = bt.run()

bt.plot() # trade execution

stats # trade statistics




Init iteration :1


Start                     2021-05-28 00:00:00
End                       2024-08-09 00:00:00
Duration                   1169 days 00:00:00
Exposure Time [%]                   99.800399
Equity Final [$]                  4063.516397
Equity Peak [$]                       10000.0
Return [%]                         -59.364836
Buy & Hold Return [%]               -10.44912
Return (Ann.) [%]                  -20.266647
Volatility (Ann.) [%]               12.326531
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -59.364836
Avg. Drawdown [%]                  -59.364836
Max. Drawdown Duration     1167 days 00:00:00
Avg. Drawdown Duration     1167 days 00:00:00
# Trades                                  232
Win Rate [%]                         28.87931
Best Trade [%]                         3.8434
Worst Trade [%]                     -2.037099
Avg. Trade [%]                    